# About

The purpose of this notebook is to provide an example of how standard DICOM objects containing annotations and evaluations of the nodules for the TCIA [LIDC-IDRI](https://wiki.cancerimagingarchive.net/display/Public/LIDC-IDRI) collection, as presented in the following article, can be used from Python.

> Fedorov A, Hancock M, Clunie D, Brochhausen M, Bona J, Kirby J, Freymann J, Pieper S, Aerts H, Kikinis R, Prior F. 2018. Standardized representation of the LIDC annotations using DICOM. PeerJ Preprints 6:e27378v1 https://doi.org/10.7287/peerj.preprints.27378

Another goal of this notebook is to validate the accuracy of the conversion from the `pylidc` representation.

## Prerequisites

This notebook assumes you have the following locally on your computer:
* Images from the TCIA LIDC-IDRI collection, see download instructions here: https://wiki.cancerimagingarchive.net/display/Public/LIDC-IDRI
* DICOM Segmentations and Structured Reporting objects (at the time of writing this, those are located separately from the images), see download instructions here: https://wiki.cancerimagingarchive.net/display/DOI/Standardized+representation+of+the+TCIA+LIDC-IDRI+annotations+using+DICOM


## Conversion consistency verification approach

Conversion verification is done by confirming that:
* each of the nodules larger than 3 mm, as queried from `pylidc`, is also encoded in DICOM;
* there is only one DICOM SEG and one DICOM SR object corresponding to a single annotation of a nodule in the DICOM dataset;
* quantitative and qualitative assessment values are consistent between the two representations.

Verification consists of evaluating two components of the dataset metadata (evaluations, measurements, referenced images) and pixel data (voxels labeled as belonging to a certain nodule).

In order to support this verification, the data first needs to be prepared for consuming it from python. Although there can be various ways to do this, our approach will utilize the following steps:
1. DICOM files are first organized using [dicomsort](https://github.com/pieper/dicomsort):

`python dicomsort.py <location of DICOM images, SEG and SR files> \
  <directory to store sorted files>/%PatientID/%StudyInstanceUID/%SeriesInstanceUID-%SeriesNumber-%Modality/%SOPInstanceUID.dcm`

2. Extract DICOM metadata into tabular form using [dcm2tables](https://github.com/QIICR/dcm2tables). The result is a set of tables that are defined by [this schema](https://app.quickdatabasediagrams.com/#/d/K6UbDf).

`python tabulate.py -s schema.qdbd -d <directory with the sorted files> -o <directory to store metadata tables>`

In [116]:
import pylidc as pl
import pandas as pd
import os, json

DICOM_PATH = "/Users/fedorov/Temp/LIDC_conversion2-sorted"
DCMTABLES_PATH = "/Users/fedorov/Temp/LIDC_conversion2-tables"


In [113]:
# read DICOM metadata tables
tablesNames = [
               'CompositeContext', # one per DICOM file, attributes that are available in every DICOM object
    
               'References',       # when applicable, references to related DICOM objects in the derived datasets
    
               'CT', 'SEG','SR',     # modality-specific attributes
    
               'SEG_Segments','SEG_SegmentFrames',  # attributes specific to DICOM Segmentations; each Segmentation
                                                    # object contains one or more segment, and each segment contains
                                                    # one or more frame (i.e., "slice") with the labeled pixels
    
               'SR1500_Measurements','SR1500_MeasurementGroups', # attributes specific to DICOM Structured Reports (SR)
               'SR1500_QualitativeEvaluations',     # qualitative assessments
    
               'Instance2File'                      # pointer from the DICOM SOPInstanceUID to the file on the filesystem
              ]
tables = {}
for t in tablesNames:
    tables[t]=pd.read_csv(os.path.join(DCMTABLES_PATH,t+'.tsv'), sep='\t', dtype=str, low_memory=False)

In [73]:
# load dictionaries
    
conceptsDictionary = {}
valuesDictionary = {}
with open("../concepts_dict.json") as cf:
  conceptsDictionary = json.load(cf)
with open("../values_dict.json") as vf:
  valuesDictionary = json.load(vf)

In [122]:
subjectsToVerify = [("LIDC-IDRI-%04i" % i) for i in range(510,511)]

# iterate over data, confirm existence of data, 
# one SEG and one SR per annotation, and consistency of numeric
# and qualitative assessment values between pylidc and DICOM representation
from decimal import *
epsilon = 1e-2

def compareNumbers(v1, v2):
  assert (abs(v1-v2) < epsilon), "Comparison threshold exceeded: %f vs %f!" % (v1, v2) 

def compareStrings(s1, s2):
  assert (s1 == s2), "Comparison strings failed: %s vs %s!" % (s1, s2) 

def compareCodes(c1, c2):
  assert (c1["CodeValue"] == c2["CodeValue"]), "CodeValue mismatch"
  assert (c1["CodeMeaning"] == c2["CodeMeaning"]), "CodeMeaning mismatch"
  assert (c1["CodingSchemeDesignator"] == c2["CodingSchemeDesignator"]), "CodingSchemeDesignator mismatch"

for s in subjectsToVerify:
  print(s)
  scans = pl.query(pl.Scan).filter(pl.Scan.patient_id == s)
  for scan in scans:
    studyUID = scan.study_instance_uid
    seriesUID = scan.series_instance_uid
    seriesDir = os.path.join(DICOM_PATH,s,studyUID,seriesUID)
    
    referencesSubset = tables["References"][tables["References"]["ReferencedSeriesInstanceUID"] == seriesUID]
    referencingUIDs = pd.unique(referencesSubset["SOPInstanceUID"])
    thoseReferenceSeries = tables["CompositeContext"][tables["CompositeContext"]["SOPInstanceUID"].isin(referencingUIDs)]
    
    segmentations = thoseReferenceSeries[thoseReferenceSeries["Modality"]=="SEG"]
    segmentsSubset = tables["SEG_Segments"][tables["SEG_Segments"]["SOPInstanceUID"].isin(referencingUIDs)]
    segments = pd.merge(segmentsSubset, segmentations, on=["SOPInstanceUID"])
        
    #print(segments["SegmentLabel"])
    
    for nCount,nodule in enumerate(scan.cluster_annotations()):
      for aCount,a in enumerate(nodule):
        
        # this is the convention used to name segments (SegmentDescription and SegmentLabel)
        segmentLabel = "Nodule "+str(nCount+1) +" - Annotation " + a._nodule_id
        # Find corresponding segment:
        #  1. find SEGs that reference the SeriesInstanceUID of the CT annotated
        #  2. find the specific SEG using SegmentLabel
        #  3. if there are multiple annotations that have the same ID, we are screwed
        segment = segments[segments["SegmentLabel"] == segmentLabel]
        assert (segment.shape[0] == 1), "More than one matching segment: %i!" % segment.shape[0]
        #print(str(segment["SOPInstanceUID"].values))
        
        segmentationInstanceUID = segment["SOPInstanceUID"].values[0]
        
        # now get measurement groups that correspond to those Structured Reports instances
        # For the specific dataset, we know there is a single measurement group per segment,
        #  and we also know there is a single segment per segmentation, so this makes life easier
        measurementGroups = tables["SR1500_MeasurementGroups"][tables["SR1500_MeasurementGroups"]["segmentationSOPInstanceUID"] == segmentationInstanceUID]
        assert (measurementGroups.shape[0] == 1), "Not one measurement group per segment: %i!" % measurementGroupsSubset.shape[0]
        structuredReportUID = measurementGroups["SOPInstanceUID"].values[0]
        
        qualEval = tables["SR1500_QualitativeEvaluations"][tables["SR1500_QualitativeEvaluations"]["SOPInstanceUID"] == structuredReportUID]
        quantEval = tables["SR1500_Measurements"][tables["SR1500_Measurements"]["SOPInstanceUID"] == structuredReportUID]        
        
        # check quantitative measurements
        assert (len(quantEval[quantEval["quantity_CodeMeaning"]=="Volume"]["value"].values) == 1), "Number of measurements is not one!"
        assert (len(quantEval[quantEval["quantity_CodeMeaning"]=="Diameter"]["value"].values) == 1), "Number of measurements is not one!"
        assert (len(quantEval[quantEval["quantity_CodeMeaning"]=="Surface area of mesh"]["value"].values) == 1), "Number of measurements is not one!"

        dcmVolume = quantEval[quantEval["quantity_CodeMeaning"]=="Volume"]["value"].values[0]
        dcmDiameter = quantEval[quantEval["quantity_CodeMeaning"]=="Diameter"]["value"].values[0]
        dcmSurface = quantEval[quantEval["quantity_CodeMeaning"]=="Surface area of mesh"]["value"].values[0]
        
        compareStrings(dcmVolume,  ("%E" % Decimal(a.volume)))
        compareStrings(dcmDiameter, ("%E" % Decimal(a.diameter)))
        compareStrings(dcmSurface, ("%E" % Decimal(a.surface_area)))
            
        dcmStr = quantEval[quantEval["quantity_CodeMeaning"]=="Volume"]["value"].values[0]
        
        # check code values
        for attribute in conceptsDictionary.keys():
          aItem = {}

          if attribute == "internalStructure" and segmentLabel == "Nodule 8 - Annotation 1708" and s == "LIDC-IDRI-0510":
            #print("  Skipping knowingly faulty item")
            continue
        
          try:
            aItem["conceptCode"] = conceptsDictionary[attribute]
            aItem["conceptValue"] = valuesDictionary[attribute][str(getattr(a, attribute))]
          
          except KeyError:            
            print(" Invalid attribute: "+attribute+': '+str(getattr(a, attribute))+" "+segmentLabel)
            
          if qualEval[qualEval["conceptCode_CodeMeaning"]==aItem["conceptCode"]["CodeMeaning"]].shape[0] != 1:
            print("  Failed to find quantitative assessment for %s in DICOM representation" % aItem["conceptCode"]["CodeMeaning"])
            continue

          #print(qualEval["conceptValue_CodeMeaning"])
          dcmItem = qualEval[qualEval["conceptCode_CodeMeaning"]==aItem["conceptCode"]["CodeMeaning"]]
          compareCodes(aItem["conceptValue"], {"CodeValue": dcmItem["conceptValue_CodeValue"].values[0], \
                                               "CodeMeaning": dcmItem["conceptValue_CodeMeaning"].values[0], \
                                               "CodingSchemeDesignator": dcmItem["conceptValue_CodingSchemeDesignator"].values[0]})
        
#      break
#    break
#  break

LIDC-IDRI-0510
Skipping knowingly faulty item
 Invalid attribute: internalStructure: 5 Nodule 8 - Annotation 1708
  Failed to find quantitative assessment for Internal structure in DICOM representation


## Visualization